# **App Store Reviews 분석**
---

![logo](./figs/App_Store_Logo.png)

## **목적**

App Store 에는 수많은 사람들이 앱에 관련해 리뷰를 달고 해당 앱에대해 평점을 매기고 있다. 많은 개발자들도 종종 리뷰를 보면서 향후 이용자와 소통을 하고 향후 앱 개발의 방향성을 탐색하고 고민하게 된다. 평소에 앱 스토어를 이용하면서 리뷰들이 정확하게 이 앱을 평가하고 있는지 혹은 앱들의 리뷰로 어떤 앱을 별점을 예측 할 수 있을 지 궁금해졌다. 그래서 이 프로젝트를 시작했다.

## **Part 1. 데이터 획득**

앱스토어 크롤링을 위해서는 우선 스토어 페이지의 고유 ID를 알아야한다. 

이는 don9z [링크](https://github.com/don9z/appstore) 분의 App Store 조회를 위한 Tool를 사용해서 한국 앱스토어의 존재하는 무료/유료 포함된 약 4541개의 url 링크를 조회하여 ID부분만 추출해냈다.

그 후 각 앱의 리뷰, 평점등 데이터는 앱스토어 고객리뷰로부터 [URL](https://itunes.apple.com/kr/rss/customerreviews/) 에서 크롤링 하여 DB에 저장했다. 10월 20일까지 작성된 리뷰 기준으로 DB 에 616,758 건의 데이터를 저장했고, 각 feature은 아래와 같다. 





|Feature Name| 설명 |
|:-:|:-|
|app_id| 앱스토어 페이지의 고유 ID| 
|app_name|앱 이름|
|review_id|리뷰 문서 ID|
|title|리뷰 문서 제목|
|author|리뷰 문서 저자|
|author_url|리뷰 url|
|version|앱 버젼|
|rating|리뷰 문서 저자가 이 앱에 대해 평가한 평점|
|review|리뷰 내용|
|category|앱 카테고리|

In [1]:
from utils import *
docs, app_id_list = get_data_json('./data/train_json.txt')

## **Part 2. 데이터 정제**

리뷰 분석을 위해서 한 문장을 최대한 작게 분할해야 한다. 그 이유는 우리는 보통 높은 단계의 사상이나 개념을 하위 단계의 요소로 세분화 하여 명확하게 정의 가능한다고 우리는 믿기 때문이다. 예를 들어 "나는 스타벅스에 가고 있어."라는 문장의 의미를 알고자 하려면 "나", "는", "스타벅스", "에", "가", "ㄴ다"의 의미를 각각 알고 이들을 조합해야 한다.

In [3]:
from konlpy.tag import Komoran
komoran = Komoran()
komoran.pos("나는 스타벅스에 간다")

[('나', 'NP'),
 ('는', 'JX'),
 ('스타벅스', 'NNP'),
 ('에', 'JKB'),
 ('가', 'VV'),
 ('ㄴ다', 'EC')]

따라서 문장을 분석하려면 문장을 구성하는 하위 개념인 단어 혹은 형태소로 자르게 된다. 보통 형태소의 기준으로 나눠서 쓴다. 그러나 여기서 문제가 생긴다. 문장 띄어쓰기나 오타가 나면 분석기가 제대로 작동하지 않는 다는 점이다. 주로 문제가 발생하는 원인은 두 가지로 확인된다.
* 오타 및 축약어는 형태소 분석기 사전에 존재 하지 않기 때문에 Unknown(혹은 NA)로 인식
* 띄어쓰기가 제대로 안될 경우 전체 구문이 Unknown(혹은 NA)로 인식, 특히 'ㅠㅠ'같은 이모티콘(단순 자음 혹은 모음의 집합)과 문장이 붙어있으면 더욱 안되는 경우가 발생한다

In [3]:
app_id_list, ratings_lists, ma_lists = read_jsonl('./data/train_jsonl_first_ma.txt')
UNK = Unknown_words()
unknowns = UNK.get_unknown_words(ma_lists)
unknowns[:10]

Extracting unknowns:: 100%|██████████| 2984/2984 [00:02<00:00, 1053.37it/s]

Unknown(중복제거): 70342, Total Unknown: 173157, Total MA: 489261


['(^(ㅇㅇ)^)',
 '업데이트가없엌ㅋㅋㅋㅋㅋㅋ',
 'ㅎ~',
 '샀어요...ㅠㅠ',
 '그래듀',
 '꼴깝한다',
 '김급식!넘좋슴다!',
 '나오긴하는뎅',
 '두번됬음.',
 '취적입니다']

### **띄어쓰기 문제**

주로 단일 자음, 모음으로 구성된 구문이 문장과 붙어 있으면 형태소 분석이 안되는 것이 띄어쓰기 문제다. 따라서 문장을 더 잘게 잘라서 해결하기로 했다.

예를 들어, "개선좀해주세요ㅠㅠ"문장을 두 글자씩 검사해서 단일 자음 혹은 모음과 완전한 글자가 조합이 되면 띄어쓰기 하는 방법으로 띄어쓰기를 하였다.

In [4]:
docs, app_id_list = get_data_json('./data/train_json.txt')
sentence = docs['333903271']['reviews'][460]
spacing = Spacing()
print('Before:', sentence[38:])
print('After:', spacing.space_jamo(sentence[38:]))

Before: 유령되요 제발 이것좀 어떻게 해주세요ㅜㅜㅜㅜㅜ진짜 찾아갈꺼에요ㅜㅜㅜ
After: 유령되요 제발 이것좀 어떻게 해주세요 ㅜㅜㅜㅜㅜ 진짜 찾아갈꺼에요 ㅜㅜㅜ


본 프로젝트에서는 우선 띄어쓰기를 실행한 후에 형태소 분석을 진행하는 방향으로 한다.

### **오타 및 축약어**

오타와 축약어를 해결 하는 방법 중 하나는 형태소 분석 전에 사전에 지정한 단어를 등록하는 것이다. 그러나 이는 매우 노가다 성격이 짙은 작업이 된다. 그리고 우리가 모든 오타와 축약어 들을 사전에 수정할 수 없기에 후처리로 넘기는 것이 최선이다. 

단어의 후처리는 형태소 분석을 진행한 후에 제대로 되지 않은 단어들을 찾아서 바꿔주는 형태가 된다. 

In [30]:
komoran.pos('업뎃')

[('업뎃', 'NA')]

"업뎃"은 "업데이트"의 축약어다. 앱스토어 리뷰에 주로 나오는 단어들을 찾아서 규칙 사전으로 기록 후에, 해당하는 단어를 찾아서 바꿔준다.

In [5]:
app_id_list, ratings_lists, ma_lists = read_jsonl('./data/train_jsonl_ma_komoran.txt')
UNK = Unknown_words()
unknowns = UNK.get_unknown_words(ma_lists)

Extracting unknowns:: 100%|██████████| 2984/2984 [00:03<00:00, 820.25it/s] 


Unknown(중복제거): 33197, Total Unknown: 180845, Total MA: 489261


In [6]:
unknowns[:10]

['나랏글사용자인',
 '부담됬었는데,',
 '업뎃합시다',
 '도당췌',
 '닶이',
 '귀찮았는뎋',
 '코프스키',
 '튕귀네요',
 '버벅버벅버벅버벅버버벅버버버벅',
 '감솨히']

In [7]:
UNK.unknown_loc_dict['업뎃합시다']

(1122, 128, 21)

In [8]:
ma_lists[1122][128]

[['일단', 'MAG'],
 ['dts', 'SL'],
 ['지원', 'NNG'],
 ['안', 'NNG'],
 ['되', 'XSV'],
 ['는', 'ETM'],
 ['것', 'NNB'],
 ['그리고', 'MAJ'],
 ['뒤', 'NNG'],
 ['로', 'JKB'],
 ['가', 'VV'],
 ['기', 'ETN'],
 ['하', 'XSV'],
 ['면', 'EC'],
 ['자막', 'NNG'],
 ['재생', 'NNG'],
 ['안', 'NNG'],
 ['되', 'XSV'],
 ['는', 'ETM'],
 ['것', 'NNB'],
 ['좀', 'MAG'],
 ['업뎃합시다', 'NA']]

In [3]:
post_process = Post_ma('./data/post_ma_pairs/komoran.txt')

Can not load file: ./data/post_ma_pairs/komoran.txt!  There is no file, please create file by update method(take option write=True)


In [ ]:
post_process.

## **Part 3. 데이터 분석**